In [71]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import lxml

req = requests.get("https://www.espncricinfo.com/records/season/team-match-results/2024-2024?trophy=89")
soup = bs(req.content)

In [73]:
summary_table = soup.find('table')
summary_df = pd.read_html(str(summary_table))[0]

summary_df.to_csv('summary.csv', index=False)

C:\Users\swara\AppData\Local\Temp\ipykernel_38572\3522204170.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(str(summary_table))[0]


# Getting Individual Matches

In [3]:
# Creating an array containing the links of scorecards of all matches
match_links = summary_table.find_all('a',)

href_list = []
for link in match_links:
  href = link.get('href')
  href_list.append(href)

links = href_list[1::2]

# First Match Scorecard:

In [4]:
link = "https://espncricinfo.com" + links[0]
req = requests.get(link)
soup = bs(req.content)

## Applying For All Matches:

In [5]:
# Function to create a dataframe from batting table

def create_batting_df(batting_table, soup):
  columns = batting_table.find('thead').find_all('th')
  column_names = [c.string for c in columns]
  table_rows = batting_table.find('tbody').find_all('tr')
  l = []
  for tr in table_rows:
    td = tr.find_all('td')
    row = [str(tr.get_text()).strip() for tr in td]
    l.append(row[:-1]) # Exclude the last element of each row
  
  df = pd.DataFrame(l, columns=column_names[:-1])  # Exclude the last column name
  df.dropna(inplace=True)
  df = df.rename(columns={df.columns[1]: 'Dissmissal'})

  # Extracting the match name
  name = soup.select('h1.ds-text-title-xs.ds-font-bold.ds-mb-2.ds-m-1')
  match_name = name[0].get_text()
  match_name = match_name.split(',')[0] 
  df['Match'] = match_name

  # Extracting the batting team name
  innings1 = soup.select('span.ds-text-title-xs.ds-font-bold.ds-capitalize')[0].get_text()
  df['innings'] = innings1
  
  return df

In [6]:
score_links = ["https://espncricinfo.com" + link for link in links]
df_batting = pd.DataFrame()
count = 0
for link in score_links:
  req = requests.get(link)
  soup = bs(req.content)
  batting_table = soup.find_all('table', class_="ci-scorecard-table")

  batting_first = batting_table[0]
  df1 = create_batting_df(batting_first, soup)

  if len(batting_table) == 2:
    batting_second = batting_table[1]
    df2 = create_batting_df(batting_second, soup)
    df_batting = pd.concat([df_batting, df1, df2], ignore_index=True)

  else:
    df_batting = pd.concat([df_batting, df1], ignore_index=True)

df_batting = df_batting.drop(columns=['SR'])


## For Bowling Scorecard:

In [7]:
link = score_links[0]
req = requests.get(link)
soup = bs(req.content)

link

'https://espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166/india-vs-south-africa-final-1415755/full-scorecard'

In [14]:
bowling_table = soup.select('table.ds-w-full.ds-table.ds-table-md.ds-table-auto')
bowling_first = bowling_table[1]
bowling_second = bowling_table[3]

In [64]:
# Function to convert bowling table to dataframe

def create_bowling_df(bowling_table, soup):
  columns = bowling_table.find('thead').find_all('th')
  column_names = [c.string for c in columns]
  table_rows = bowling_table.find('tbody').find_all('tr')
  l = []
  for tr in table_rows:
    td = tr.find_all('td')
    row = [str(tr.get_text()).strip() for tr in td]

    if len(row) != len(column_names):
      row = row[:-1]

    l.append(row)

  df = pd.DataFrame(l, columns=column_names)
  df.dropna(inplace=True)

  # Extracting the match name
  name = soup.select('h1.ds-text-title-xs.ds-font-bold.ds-mb-2.ds-m-1')
  match_name = name[0].get_text()
  match_name = match_name.split(',')[0] 
  df['Match'] = match_name

  # Extracting the batting team name
  innings1 = soup.select('span.ds-text-title-xs.ds-font-bold.ds-capitalize')[0].get_text()
  df['innings'] = innings1
  
  return df



In [65]:
req = requests.get(link)
soup = bs(req.content)
bowling_table = soup.select('table.ds-w-full.ds-table.ds-table-md.ds-table-auto')

bowling_first = bowling_table[1]
bowling_second = bowling_table[3]

columns = bowling_first.find('thead').find_all('th')
column_names = [c.string for c in columns]
table_rows = bowling_first.find('tbody').find_all('tr')
l = []
for tr in table_rows:
  td = tr.find_all('td')
  row = [str(tr.get_text()).strip() for tr in td]
  
  if len(row) != len(column_names):
    column_names.insert(1, np.nan)

  l.append(row)

In [66]:
df_bowling = pd.DataFrame()
count = 0

for link in score_links:
  req = requests.get(link)
  soup = bs(req.content)
  bowling_table = soup.select('table.ds-w-full.ds-table.ds-table-md.ds-table-auto')
  print(count)

  bowling_first = bowling_table[1]
  df1 = create_bowling_df(bowling_first, soup)


  if len(bowling_table) == 4:
    bowling_second = bowling_table[3]
    df2 = create_bowling_df(bowling_second, soup)
    df_bowling = pd.concat([df_bowling, df1, df2], ignore_index=True)

  else:
    df_bowling = pd.concat([df_bowling, df1], ignore_index=True)

  count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [67]:
df_bowling

,Bowling,O,M,R,W,ECON,0s,4s,6s,WD,NB,Match,innings
0,Marco Jansen,4,0,49,1,12.25,4,5,2,1,1,India vs South Africa,India
1,Keshav Maharaj,3,0,23,2,7.66,10,3,1,0,0,India vs South Africa,India
2,Kagiso Rabada,4,0,36,1,9.00,8,2,2,2,0,India vs South Africa,India
3,Aiden Markram,2,0,16,0,8.00,1,0,1,0,0,India vs South Africa,India
4,Anrich Nortje,4,0,26,2,6.50,7,2,0,2,0,India vs South Africa,India
...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,Jeremy Gordon,3,0,44,0,14.66,7,4,3,3,2,U.S.A. vs Canada,Canada
591,Dilon Heyliger,3,0,19,1,6.33,9,0,1,2,0,U.S.A. vs Canada,Canada
592,Saad Bin Zafar,4,0,42,0,10.50,9,2,4,0,0,U.S.A. vs Canada,Canada
593,Nikhil Dutta,2.4,0,41,1,15.37,5,2,4,2,1,U.S.A. vs Canada,Canada


In [74]:
df_bowling.to_csv("bowling.csv", index=False)
df_batting.to_csv("batting.csv", index=False)